# Neural Network Assignment

In [65]:
import warnings
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, chi2

In [7]:
def CrossValidacaoEstratificada(dataset, y, folds=4, seed=42):
    np.random.seed(seed)
    npY = np.array(y)
    fold_classes = list()

    #construindo as estruturas
    dataset_fold = list()
    for i in range(folds):
        dataset_fold.append(list())
    fold_atual = 0
    unicos = np.unique(npY)
    for i in range(len(unicos)):
        # cria uma lista das classe e os valores os índices(posições) delas no vetor y
        fold_classes.append(np.where(npY == unicos[i])[0].tolist())
        
        while len(fold_classes[i])>0:
            # sorteia um elemento do vetor de elementos da mesma classe
            if (fold_atual >= folds):
                fold_atual = 0
            index_elemento = np.random.randint(len(fold_classes[i]))
            index = fold_classes[i].pop(index_elemento)
            # Adiciona o elemento sorteado no bucket correspondente
            dataset_fold[fold_atual].append(index)
            fold_atual = fold_atual + 1
    for i in range(len(dataset_fold)):
        print("dataset_fold[" + str(i) + "]: " + str(len(dataset_fold[i])))
    return dataset_fold

In [10]:
def load_wine(normalizar):
    # função que carrega os dados do dataset wine, e remove a classe '3' para se tornar um dataset binário
    names = ['class','alcohol','malic_acid','ash','alcalinity_of_ash','magnesium','total_phenols','flavanoids','nonflavanoid_phenols'
              ,'proanthocyanins','color_intensity','hue','OD280_OD315_of_diluted_wines','proline']
    data = pd.read_csv('../Data/wine.data', names=names)
    data_binario = data.loc[data["class"] != 3,:]
    
    if normalizar:
        scaler = StandardScaler()
        scaler.fit(data_binario.drop('class', axis=1))
        X = pd.DataFrame(scaler.transform(data_binario.drop('class', axis=1)), columns=names[1:])
    else:
        X = data_binario.drop('class', axis=1)
    y = data_binario.loc[:,'class']
    return X, y

In [76]:
def metricas(classificador, X, y, folds, seed=42):
    np.random.seed(seed)
    recall = cross_val_score(classificador, X, y, cv=folds, scoring='recall')
    precision = cross_val_score(classificador, X, y, cv=folds, scoring='precision')
    accuracy = cross_val_score(classificador, X, y, cv=folds, scoring='accuracy')
    return (np.mean(recall), np.mean(precision), np.mean(accuracy))

In [77]:
X, y = load_wine(normalizar='True')
dataset_indexes = CrossValidacaoEstratificada(X, y, folds=4)

dataset_fold[0]: 33
dataset_fold[1]: 33
dataset_fold[2]: 32
dataset_fold[3]: 32


In [78]:
# Training the MLP
# we will use the sgd representing the stochastic gradient descent
mlp = MLPClassifier(solver='sgd', alpha=1e-5,
                    hidden_layer_sizes=(5, 2), random_state=1)

In [79]:
warnings.filterwarnings('ignore')
revocacao, precisao, acuracia = metricas(mlp, X, y, folds=4)

In [80]:
print("Revocação: " + str(revocacao))
print("Precisão: " + str(precisao))
print("Acurácia: " + str(acuracia))

Revocação: 0.8666666666666667
Precisão: 0.9539215686274509
Acurácia: 0.9161779081133921


In [81]:
def training_MLP(X, y, folds, layers, neurons, learning_rate):
    for layer in range(len(layers)):
        for neuron in range(len(neurons)):
            print("\nMLP com " + str(layers[layer]) + " camadas ocultas e " + str(neurons[neuron]) + " por camada:\n")
            for l_rate in range(len(learning_rate)):
                print("Learning rate " + str(learning_rate[l_rate]) + ":\n")
                mlp = MLPClassifier(solver='sgd', alpha=learning_rate[l_rate],
                        hidden_layer_sizes=(neurons[neuron], layers[layer]), random_state=1)
                revocacao, precisao, acuracia = metricas(mlp, X, y, folds=4)
                print("Revocação: " + str(revocacao))
                print("Precisão: " + str(precisao))
                print("Acurácia: " + str(acuracia))
                print("\n")

In [82]:
layers = [1, 2, 3]
neurons = [2, 5, 10]
learning_rate = [0.1, 0.05, 0.01] # learning rates
folds = 4
training_MLP(X, y, folds, layers, neurons, learning_rate)


MLP com 1 camadas ocultas e 2 por camada:

Learning rate 0.1:

Revocação: 0.0
Precisão: 0.0
Acurácia: 0.5461876832844574


Learning rate 0.05:

Revocação: 0.0
Precisão: 0.0
Acurácia: 0.5461876832844574


Learning rate 0.01:

Revocação: 0.0
Precisão: 0.0
Acurácia: 0.5461876832844574



MLP com 1 camadas ocultas e 5 por camada:

Learning rate 0.1:

Revocação: 0.9488095238095238
Precisão: 0.5096153846153846
Acurácia: 0.5613391984359726


Learning rate 0.05:

Revocação: 0.9488095238095238
Precisão: 0.5096153846153846
Acurácia: 0.5613391984359726


Learning rate 0.01:

Revocação: 0.9488095238095238
Precisão: 0.5096153846153846
Acurácia: 0.5613391984359726



MLP com 1 camadas ocultas e 10 por camada:

Learning rate 0.1:

Revocação: 0.0
Precisão: 0.0
Acurácia: 0.5461876832844574


Learning rate 0.05:

Revocação: 0.0
Precisão: 0.0
Acurácia: 0.5461876832844574


Learning rate 0.01:

Revocação: 0.0
Precisão: 0.0
Acurácia: 0.5461876832844574



MLP com 2 camadas ocultas e 2 por camada:

Learnin

## Aplicando o PCA de 1 componente que correponde a mais de 95% da variancia do dataset

In [63]:
pca = PCA(n_components=1)
X_PCA = pca.fit_transform(X)

In [64]:
layers = [1, 2, 3]
neurons = [2, 5, 10]
learning_rate = [0.1, 0.05, 0.01] # learning rates
folds = 4
training_MLP(X_PCA, y, folds, layers, neurons, learning_rate)


MLP com 1 camadas ocultas e 2 por camada:

Learning rate 0.1:

Revocação: 1.0
Precisão: 0.4538123167155425
Acurácia: 0.4538123167155425


Learning rate 0.05:

Revocação: 1.0
Precisão: 0.4538123167155425
Acurácia: 0.4538123167155425


Learning rate 0.01:

Revocação: 1.0
Precisão: 0.4538123167155425
Acurácia: 0.4538123167155425



MLP com 1 camadas ocultas e 5 por camada:

Learning rate 0.1:

Revocação: 0.06785714285714287
Precisão: 0.75
Acurácia: 0.5769794721407625


Learning rate 0.05:

Revocação: 0.06785714285714287
Precisão: 0.75
Acurácia: 0.5769794721407625


Learning rate 0.01:

Revocação: 0.06785714285714287
Precisão: 0.75
Acurácia: 0.5769794721407625



MLP com 1 camadas ocultas e 10 por camada:

Learning rate 0.1:

Revocação: 0.15119047619047618
Precisão: 1.0
Acurácia: 0.6148582600195504


Learning rate 0.05:

Revocação: 0.15119047619047618
Precisão: 1.0
Acurácia: 0.6148582600195504


Learning rate 0.01:

Revocação: 0.15119047619047618
Precisão: 1.0
Acurácia: 0.6148582600195504

### Aplicando o método de seleção de caracteristicas SelectKBest usando como função de score o teste chi^2, podendo assim selectinar as características mais independentes entre si.

In [70]:
X, y = load_wine(normalizar = False)
#Encontrar as 5 melhores características do dataset
bestfeatures=SelectKBest(score_func=chi2,k=5)
fit = bestfeatures.fit(X,y)
dfscores=pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)

#Concatenando 2 dataframes para melhor visualização

featureScores=pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns=['Specs','Score']
print(featureScores.nlargest(10,'Score'))


                           Specs         Score
12                       proline  14497.066903
9                color_intensity     45.797138
4                      magnesium     44.833856
3              alcalinity_of_ash     17.573073
6                     flavanoids     10.517824
0                        alcohol      5.350222
5                  total_phenols      4.316162
11  OD280_OD315_of_diluted_wines      1.512945
8                proanthocyanins      1.330983
2                            ash      0.611822


In [74]:
# Selecionando as TOP 7
X_selected = X[['proline', 'color_intensity', 'magnesium', 'alcalinity_of_ash', 'flavanoids', 'alcohol']]
# normalizando o dataset
scaler = StandardScaler()
scaler.fit(X_selected)
X_selected_normalized = scaler.transform(X_selected)

In [75]:
layers = [1, 2, 3]
neurons = [2, 5, 10]
learning_rate = [0.1, 0.05, 0.01] # learning rates
folds = 4
training_MLP(X_selected_normalized, y, folds, layers, neurons, learning_rate)


MLP com 1 camadas ocultas e 2 por camada:

Learning rate 0.1:

Revocação: 1.0
Precisão: 0.4538123167155425
Acurácia: 0.4538123167155425


Learning rate 0.05:

Revocação: 1.0
Precisão: 0.4538123167155425
Acurácia: 0.4538123167155425


Learning rate 0.01:

Revocação: 1.0
Precisão: 0.4538123167155425
Acurácia: 0.4538123167155425



MLP com 1 camadas ocultas e 5 por camada:

Learning rate 0.1:

Revocação: 0.9333333333333333
Precisão: 0.9341299019607843
Acurácia: 0.9393939393939394


Learning rate 0.05:

Revocação: 0.9333333333333333
Precisão: 0.9341299019607843
Acurácia: 0.9393939393939394


Learning rate 0.01:

Revocação: 0.9333333333333333
Precisão: 0.9341299019607843
Acurácia: 0.9393939393939394



MLP com 1 camadas ocultas e 10 por camada:

Learning rate 0.1:

Revocação: 0.0
Precisão: 0.0
Acurácia: 0.5461876832844574


Learning rate 0.05:

Revocação: 0.0
Precisão: 0.0
Acurácia: 0.5461876832844574


Learning rate 0.01:

Revocação: 0.0
Precisão: 0.0
Acurácia: 0.5461876832844574



MLP c